### 서울시내 공원 현황 데이터 시각화

In [1]:
import pandas as pd
import folium
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [2]:
df = pd.read_csv('data/서울시공원현황.csv', encoding='euc-kr')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            132 non-null    int64  
 1   공원명           132 non-null    object 
 2   공원개요          131 non-null    object 
 3   면적            129 non-null    object 
 4   개원일           122 non-null    object 
 5   주요시설          124 non-null    object 
 6   주요식물          88 non-null     object 
 7   안내도           102 non-null    object 
 8   오시는길          115 non-null    object 
 9   이용시참고사항       120 non-null    object 
 10  이미지           132 non-null    object 
 11  지역            131 non-null    object 
 12  공원주소          132 non-null    object 
 13  관리부서          132 non-null    object 
 14  전화번호          132 non-null    object 
 15  X좌표(GRS80TM)  122 non-null    float64
 16  Y좌표(GRS80TM)  122 non-null    float64
 17  X좌표(WGS84)    131 non-null    float64
 18  Y좌표(WGS84)    131 non-null    

In [3]:
df = df[['공원명', '면적', '공원주소', 'X좌표(WGS84)', 'Y좌표(WGS84)']]
df.dropna(subset=['면적', 'X좌표(WGS84)'],inplace=True)
df.head(4)

,공원명,면적,공원주소,X좌표(WGS84),Y좌표(WGS84)
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,길동생태공원,80683㎡,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,서울대공원,9132690m²,경기도 과천시 대공원광장로 102,127.019846,37.426449
3,서울숲,480994㎡,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072


In [4]:
# 이상한 데이터 삭제
df.drop([6], inplace=True)

In [5]:
# 시작할 때 있는 '총' 지우기
df.면적 = df.면적.str.replace('총', '')

In [6]:
# 앞에 있는 숫자만 추출하기 - 구분자 ㎡, ㎥, m
# 구분자가 여러개인 경우 split()
import re
re.split('[,.]', 'a,b,c')

['a', 'b', 'c']

- 면적 데이터 전처리

In [7]:
# df.면적 = df.면적.apply(lambda x: re.split('[㎡㎥m]', x)[0])
# df.면적 = df.면적.astype(float).astype(int)
df.면적 = df.면적.str.extract(r'(\d+)').astype(int)
df.head(5)

,공원명,면적,공원주소,X좌표(WGS84),Y좌표(WGS84)
0,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,서울대공원,9132690,경기도 과천시 대공원광장로 102,127.019846,37.426449
3,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072
4,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805


- 공원 크기에 따른 분류

In [8]:
# 기준: 소형 < 100000, 중형 < 1000000, else 대형
criteria = [0, 100000, 1000000, 100000000]
label = ['소형', '중형', '대형']            # 분류 기준
size_info = [3, 7, 15]                      # circle marker의 크기
df['분류'] = pd.cut(df.면적, criteria, labels=label)
df['size'] = pd.cut(df.면적, criteria, labels=size_info)
df

,공원명,면적,공원주소,X좌표(WGS84),Y좌표(WGS84),분류,size
0,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140,대형,15
1,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394,소형,3
2,서울대공원,9132690,경기도 과천시 대공원광장로 102,127.019846,37.426449,대형,15
3,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072,중형,7
4,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805,대형,15
...,...,...,...,...,...,...,...
126,서일대뒷산공원,631021,서울특별시 중랑구 면목동 산19,127.098782,37.587716,중형,7
127,용마도시자연공원(사가정공원),109635,서울특별시 중랑구 면목7동 산50,127.095997,37.579762,중형,7
128,문화비축기지,140022,서울특별시 마포구 증산로 87,126.893246,37.571718,중형,7
129,경춘선숲길,211392,노원구 공릉2동 산 82-2,127.076482,37.627077,중형,7


In [9]:
df.to_csv('data/공원(preprocessed)', index=False)

In [10]:
park = folium.Map([37.55, 126.98], zoom_start=11)

for i in df.index:
    folium.CircleMarker(
        location=[df['Y좌표(WGS84)'][i], df['X좌표(WGS84)'][i]],
        radius=int(df['size'][i]),
        popup=folium.Popup(df.공원주소[i], max_width=200), fill=True, color='black',
        tooltip=df.공원명[i]
        ).add_to(park)
    
title_html = "<h3 align='center' style='font-size:20px'>서울 주요 공원</h3>"
park.get_root().html.add_child(folium.Element(title_html))
park
